In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from tqdm import tqdm


In [3]:
class AgeDataset(Dataset):
    def __init__(self, df, data_path, transform=None):
        self.df = df
        self.data_path = data_path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_name = os.path.join(self.data_path, self.df.iloc[index, 0])
        image = Image.open(img_name).convert('RGB')
        age = self.df.iloc[index, 1]
        if self.transform:
            image = self.transform(image)
        return image, age

# Paths to dataset and annotations
data_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train'
annotations_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train.csv'

# Load annotations
annotations = pd.read_csv(annotations_path)

# Split into train and validation sets
train_annotations, val_annotations = train_test_split(annotations, test_size=0.2, random_state=42)

# Calculate class weights for the WeightedRandomSampler
age_counts = train_annotations['age'].value_counts().to_dict()
train_annotations['weight'] = train_annotations['age'].apply(lambda x: 1.0 / age_counts[x])

# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = AgeDataset(train_annotations, data_path, transform=train_transform)
val_dataset = AgeDataset(val_annotations, data_path, transform=val_transform)

# Create DataLoaders with WeightedRandomSampler for the training set
sampler = WeightedRandomSampler(train_annotations['weight'].values, int(len(train_annotations['weight'])))
train_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)


In [4]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the ResNet model with fine-tuning, including batch normalization layers
model = models.resnet50(pretrained=True)
for name, child in model.named_children():
    if isinstance(child, nn.BatchNorm2d):
        for param in child.parameters():
            param.requires_grad = True
    elif name in ['layer4', 'fc']:
        for param in child.parameters():
            param.requires_grad = True

# Replace the final layer with a linear layer for age prediction
model.fc = nn.Linear(model.fc.in_features, 1)
model = model.to(device)

# Define the loss function and the optimizer
criterion = nn.L1Loss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.1, min_lr=1e-6, verbose=True)

# Training and validation loops with early stopping
num_epochs = 15
best_val_loss = float('inf')
early_stopping_counter = 0
early_stopping_patience = 3

for epoch in range(num_epochs):
    # Training loop
    model.train()
    total_train_loss = 0
    for images, ages in tqdm(train_loader):
        images = images.to(device)
        ages = ages.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, ages)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    # Validation loop
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for val_images, val_ages in val_loader:
            val_images = val_images.to(device)
            val_ages = val_ages.float().unsqueeze(1).to(device)
            val_outputs = model(val_images)
            val_loss = criterion(val_outputs, val_ages)
            total_val_loss += val_loss.item()
            
    # Calculate average losses
    avg_train_loss = total_train_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(val_loader)

    # Print epoch stats
    print(f'Epoch {epoch+1}/{num_epochs} - Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}')

    # Learning rate scheduling based on validation loss
    scheduler.step(avg_val_loss)

    # Early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 153MB/s] 
100%|██████████| 267/267 [01:49<00:00,  2.44it/s]


Epoch 1/15 - Training Loss: 20.6426, Validation Loss: 7.3861


100%|██████████| 267/267 [01:42<00:00,  2.60it/s]


Epoch 2/15 - Training Loss: 7.5629, Validation Loss: 7.5221


100%|██████████| 267/267 [01:42<00:00,  2.60it/s]


...

Epoch 11/15 - Training Loss: 4.4789, Validation Loss: 5.2163


100%|██████████| 267/267 [01:41<00:00,  2.63it/s]


Epoch 12/15 - Training Loss: 4.3056, Validation Loss: 5.4654


100%|██████████| 267/267 [01:42<00:00,  2.61it/s]


Epoch 13/15 - Training Loss: 4.1478, Validation Loss: 5.6322
Early stopping triggered.


In [5]:
test_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/test'
test_ann = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv'

# No transformations are applied here as it's the test set
test_dataset = AgeDataset(pd.read_csv(test_ann), test_path, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

# Define the predict function
@torch.no_grad()
def predict(loader, model):
    model.eval()
    predictions = []
    for images, _ in loader:
        images = images.to(device)
        outputs = model(images)
        predictions.extend(outputs.view(-1).cpu().detach().numpy())
    return predictions

# Run predictions
predictions = predict(test_loader, model)

# Prepare the submission file
submission = pd.read_csv(test_ann)
submission['age'] = predictions
submission.to_csv('/kaggle/working/submission.csv', index=False)
